Step 1. Build a CaF₂ Cluster with ASE

In [1]:
from ase import Atoms
from ase.build import make_supercell
from ase.io import write
from ase.build import bulk
from ase.cluster import wulff_construction
from ase.io import write

# Build CaF2 fluorite unit cell
caf2 = bulk('CaF2', 'fluorite', a=5.46)  # experimental lattice ~5.46 Å

# Cut out a small cluster (~1 nm) with CaF2 stoichiometry
# (Alternative: just take supercell and slice central atoms)
cluster = caf2.repeat((2,2,2))  # 2x2x2 supercell
cluster.center(vacuum=6.0)

write("caf2_cluster.xyz", cluster)
print("Cluster with", len(cluster), "atoms written to caf2_cluster.xyz")

Cluster with 24 atoms written to caf2_cluster.xyz


Step 2. Build a CaF₂:Er Cluster with ASE

In [2]:
from ase.build import bulk
from ase.io import write
import numpy as np

# Step 1: Build CaF2 fluorite cell
caf2 = bulk('CaF2', 'fluorite', a=5.46)

# Step 2: Make a small cluster (supercell with vacuum)
cluster = caf2.repeat((2,2,2))
cluster.center(vacuum=6.0)

# Step 3: Substitute one Ca -> Er to get Ca3:Er ratio in cluster
symbols = cluster.get_chemical_symbols()
ca_indices = [i for i, s in enumerate(symbols) if s == "Ca"]

if len(ca_indices) < 4:
    raise RuntimeError("Not enough Ca atoms to make Ca3:Er substitution!")

# Replace the 4th Ca with Er
symbols[ca_indices[3]] = "Er"
cluster.set_chemical_symbols(symbols)

# Step 4: Save
write("ca3f_er_cluster.xyz", cluster)
print("Cluster with", len(cluster), "atoms written to ca3f_er_cluster.xyz")

Cluster with 24 atoms written to ca3f_er_cluster.xyz


3. Simulate

In [ ]:
import os, time, inspect, sys
import numpy as np
from ase.io import read, write
from ase.optimize import LBFGS
from gpaw import GPAW, FermiDirac
from gpaw.tddft import TDDFT
from gpaw.mixer import Mixer, MixerSum, MixerDif
from gpaw.poisson import PoissonSolver
from gpaw import GPAW
from gpaw.lrtddft import LrTDDFT
import matplotlib.pyplot as plt
from scipy.ndimage import gaussian_filter1d

# --- DeltaKick ---
class DeltaKick:
    def __init__(self, strength=0.001, direction=(1, 0, 0)):
        self._strength = strength
        self.direction = np.array(direction, dtype=float)
        self.applied = False

    def strength(self, time):
        if not self.applied and abs(time) < 1e-8:
            self.applied = True
            return self._strength * self.direction
        return np.zeros(3)


# --- Step 1: Safer LCAO Relax ---
def relax_lcao(xyz_file):
    """
    Relax cluster geometry in LCAO mode with safer SCF settings.
    Uses gentle mixing and looser criteria to improve convergence.
    """
    name = os.path.splitext(xyz_file)[0]
    gpw_lcao = f"{name}_lcao.gpw"

    # Load system
    atoms = read(xyz_file)
    atoms.center(vacuum=10.0)

    # Calculator with cluster-friendly SCF settings
    calc = GPAW(mode="lcao",
                basis="dzp",
                xc="PBE",
                occupations=FermiDirac(0.1),   # slightly larger smearing
                kpts=(1, 1, 1),
                mixer={'backend': 'pulay',
                       'beta': 0.05,    # gentler mixing
                       'nmaxold': 5,    # fewer densities kept
                       'weight': 50.0}, # softer damping
                convergence={'energy': 1e-2, 'density': 1e-2},
                txt=f"{name}_lcao.log")
    atoms.calc = calc

    # Initial SCF to stabilize density before relaxation
    print("Running initial SCF to stabilize density...")
    atoms.get_potential_energy()

    # Geometry optimization (looser fmax first)
    dyn = LBFGS(atoms, logfile=f"{name}_lcao_opt.log")
    dyn.run(fmax=0.15, steps=200)   # relaxed force tolerance

    # Save results
    calc.write(gpw_lcao, mode="all")
    write(f"{name}_lcao_relaxed.xyz", atoms)
    print(f"✔ {name}: saved {gpw_lcao}")
    return gpw_lcao


# --- Step 2: FD Restart (gentle first) ---
def groundstate_fd(gpw_lcao, virt_buffer=20, h=0.30):
    name = gpw_lcao.replace("_lcao.gpw", "_fd")
    gpw_fd = f"{name}.gpw"
    if os.path.exists(gpw_fd):
        print(f"Using cached {gpw_fd}")
        return gpw_fd

    lcao_calc = GPAW(gpw_lcao)
    n_occ = int(np.ceil(lcao_calc.get_number_of_electrons() / 2.0))
    n_bands = n_occ + virt_buffer

    calc = GPAW(gpw_lcao,
                mode="fd",
                h=h,
                xc="PBE",
                occupations=FermiDirac(0.03),
                nbands=n_bands,
                poissonsolver=PoissonSolver('fd', eps=1e-12),
                mixer={"beta": 0.05, "nmaxold": 10, "weight": 100},
                convergence={"energy": 3e-4, "density": 2e-4, "eigenstates": 4},
                symmetry={"point_group": False},
                txt=f"{name}.log")
    calc.get_potential_energy()
    calc.write(gpw_fd, mode="all")
    return gpw_fd

# --- Step 3: Linear-Response TDDFT (LrTDDFT) ---
def run_lrtddft(gpw_fd, prefix="system", emax=6.0, sigma=0.1):
    """Run LR-TDDFT on all available states (no nroots), with live progress."""
    print(f"\n=== Running LrTDDFT on {gpw_fd} ===")
    t_start = time.time()
    calc = GPAW(gpw_fd)

    if "nstates" in inspect.signature(LrTDDFT.__init__).parameters:
        print("Detected GPAW ≥24.x → using all KS transitions (no nstates limit)")
        lr = LrTDDFT(calc)      # all transitions
        lr.diagonalize()        # all roots
    else:
        print("Detected GPAW ≤22.x → diagonalize() with no nroots → all roots")
        lr = LrTDDFT(calc)
        lr.diagonalize()        # all roots

    # --- Progress + extraction ---
    energies_eV, osc_strengths = []
    energies_eV, osc_strengths = [], []
    n_exc = len(lr)
    print(f"\nExtracting {n_exc} excitations (plot filter: E ≤ {emax:.2f} eV)\n")
    t0 = time.time()

    for i, exc in enumerate(lr, start=1):
        e = exc.get_energy() * 27.2114  # Hartree → eV
        f = np.linalg.norm(exc.get_oscillator_strength())

        if e <= emax:
            energies_eV.append(e)
            osc_strengths.append(f)

        # Progress bar + ETA
        elapsed = time.time() - t0
        avg = elapsed / i
        eta = avg * (n_exc - i)
        bar = "█" * int((i / n_exc) * 20)
        sys.stdout.write(
            f"\r[{bar:<20}] {i}/{n_exc}  "
            f"E={e:6.3f} eV  f={f:6.4f}  ETA: {eta/60:.1f} min"
        )
        sys.stdout.flush()
    print("\nDone extracting excitations.\n")

    # --- Save stick spectrum ---
    out_csv = f"{prefix}_sticks.csv"
    np.savetxt(out_csv,
               np.column_stack([energies_eV, osc_strengths]),
               delimiter=",",
               header="Energy (eV),OscStrength",
               comments="")

    # --- Broadened spectrum (plot only up to emax) ---
    x = np.linspace(0, emax + 0.5, 1000)
    y = np.zeros_like(x)
    for e, f in zip(energies_eV, osc_strengths):
        y[np.argmin(np.abs(x - e))] += f
    y_s = gaussian_filter1d(y, sigma * 100)

    out_broadened = f"{prefix}_spectrum.csv"
    np.savetxt(out_broadened, np.column_stack([x, y_s]),
               delimiter=",", header="Energy (eV),Intensity", comments="")

    plt.figure(figsize=(6, 4))
    plt.plot(x, y_s, label=f"{prefix}")
    plt.scatter(energies_eV, osc_strengths, c="red", s=12, marker="x", label="Excitations")
    plt.xlabel("Energy (eV)")
    plt.ylabel("Oscillator Strength (arb. units)")
    plt.legend()
    plt.tight_layout()
    plt.savefig(f"{prefix}_spectrum.png", dpi=300)
    plt.close()

    # --- Descriptors for QML ---
    I_area = np.trapz(y_s, x) + 1e-12
    I_norm = y_s / I_area
    m1 = np.trapz(x * I_norm, x)
    m2 = np.trapz(((x - m1) ** 2) * I_norm, x)
    peak_E = x[np.argmax(y_s)]
    features = np.array([[m1, m2, peak_E, I_area]])
    out_features = f"{prefix}_features.csv"
    np.savetxt(out_features, features, delimiter=",",
               header="m1_eV,m2_eV2,peak_eV,area", comments="")

    print(f"TDDFT runtime: {(time.time() - t_start)/60:.2f} min")
    print(f"Files: {out_csv}, {out_broadened}, {out_features}, {prefix}_spectrum.png\n")
    return out_csv, out_broadened, out_features, f"{prefix}_spectrum.png"

# ---------------------------
# Master Pipeline
# ---------------------------
import os

def run_pipeline(xyz_file):
    prefix = xyz_file.replace(".xyz", "")
    gpw_lcao = f"{prefix}_lcao.gpw"
    gpw_fd = f"{prefix}_fd.gpw"
    spectrum_csv = f"{prefix}_spectrum.csv"
    spectrum_png = f"{prefix}_spectrum.png"

    print(f"\n=== Starting pipeline for {prefix} ===\n")

    # --- Step 1: Relax geometry (LCAO mode) ---
    if os.path.exists(gpw_lcao):
        print(f"✔ Using cached LCAO: {gpw_lcao}")
    else:
        print(f"Running new LCAO relaxation for {prefix}...")
        gpw_lcao = relax_lcao(xyz_file)

    # --- Step 2: Compute or reuse FD ground state ---
    if os.path.exists(gpw_fd):
        print(f"✔ Using cached FD: {gpw_fd}")
    else:
        print(f"Running new FD ground-state for {prefix}...")
        gpw_fd = groundstate_fd(gpw_lcao)

    # --- Step 3: Run TDDFT (skip if already done) ---
    if os.path.exists(spectrum_csv):
        print(f"✔ TDDFT already completed for {prefix} — skipping.\n")
        return spectrum_csv, spectrum_png
    else:
        print(f"Running fresh LR-TDDFT for {prefix}...")
        spectrum_csv, spectrum_broad, features_csv, spectrum_png = run_lrtddft(
            gpw_fd,
            prefix=prefix
        )
        print(f"TDDFT complete: {spectrum_csv}, {spectrum_png}\n")
        return spectrum_csv, spectrum_png

# Run on pristine cluster
spectrum_files = run_pipeline("caf2_cluster.xyz")

# Run on Er-doped cluster
# spectrum_files = run_pipeline("caf2_er_cluster.xyz")

RPA estimated time left 13h36m21s
RPA kss[3]= # <KSSingle> 0->97 0(0) eji=43.933[eV] (-9.57075e-06,5.03766e-06,3.87248e-06)
RPA estimated time left 13h12m4s
RPA kss[4]= # <KSSingle> 0->98 0(0) eji=43.977[eV] (-0.0119387,0.00592942,0.00600315)
RPA estimated time left 16h22m31s
RPA kss[5]= # <KSSingle> 0->99 0(0) eji=43.9771[eV] (-3.84178e-05,0.0103556,-0.0103181)
RPA estimated time left 15h45m19s
RPA kss[6]= # <KSSingle> 0->100 0(0) eji=44.3364[eV] (-0.00200066,-0.00201333,-0.0020153)
RPA estimated time left 15h9m4s
RPA kss[7]= # <KSSingle> 0->101 0(0) eji=44.5922[eV] (-0.00559275,-0.0055356,-0.00554099)
RPA estimated time left 16h41m19s
RPA kss[8]= # <KSSingle> 0->102 0(0) eji=44.6367[eV] (3.75005e-05,-0.00056945,0.000532918)
RPA estimated time left 16h31m38s
RPA kss[9]= # <KSSingle> 0->103 0(0) eji=44.6392[eV] (-0.00111984,0.000453376,0.000511986)
RPA estimated time left 15h33m4s
RPA kss[10]= # <KSSingle> 0->104 0(0) eji=44.7388[eV] (0.000152656,-0.00152074,0.00136584)
RPA estimated t